# VacationPy
----

#### Observable Trends and Insights
By creating an interactive geospatial, Weather information, heat maps, and visualization through Google API, and other similar API applications you are able to discover possible socio-economic values buried inside monotonously large datasets.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load the csv exported in Part I to a DataFrame

weather_csv_file = "../output_data/city_weather_data.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()


,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kaitangata,-46.2817,169.8464,57.99,65.0,3.0,8.01,NZ,1.613080e+09
1,albany,42.6001,-73.9662,27.00,41.0,63.0,5.01,US,1.613080e+09
2,rikitea,-23.1203,-134.9692,79.09,71.0,82.0,8.52,PF,1.613080e+09
3,podor,16.6527,-14.9587,82.40,28.0,0.0,2.30,MR,1.613081e+09
4,hobart,-42.8794,147.3294,64.99,48.0,20.0,17.27,AU,1.613081e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "lng"]].astype(float)

# Create a humidity Heatmap layer
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
# Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,chabahar,25.2919,60.6430,70.34,46.0,0.0,5.64,IR,1.613081e+09
29,negombo,7.2083,79.8358,71.60,78.0,0.0,6.91,LK,1.613080e+09
50,saint-philippe,-21.3585,55.7679,78.80,65.0,0.0,5.75,RE,1.613080e+09
62,poum,-20.2333,164.0167,79.05,72.0,0.0,8.90,NC,1.613081e+09
218,sao filipe,14.8961,-24.4956,72.07,67.0,0.0,9.91,CV,1.613081e+09
249,vallenar,-28.5708,-70.7581,79.47,38.0,0.0,7.20,CL,1.613081e+09
393,am timan,11.0297,20.2827,73.33,18.0,0.0,7.11,TD,1.613081e+09
397,salalah,17.0151,54.0924,71.60,53.0,0.0,4.61,OM,1.613081e+09
400,bara,10.3744,10.7288,73.49,19.0,0.0,7.74,NG,1.613081e+09
489,maceio,-9.6658,-35.7353,77.00,83.0,0.0,5.75,BR,1.613081e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 22: chabahar.
Closest hotel is هتل گدروشیا.
------------
Retrieving Results for Index 29: negombo.
Closest hotel is Hotel Sunset Beach.
------------
Retrieving Results for Index 50: saint-philippe.
Closest hotel is Chambres d'hôte "La Trinité".
------------
Retrieving Results for Index 62: poum.
Closest hotel is Gîte kajeon.
------------
Retrieving Results for Index 218: sao filipe.
Closest hotel is Tortuga B&B.
------------
Retrieving Results for Index 249: vallenar.
Closest hotel is Humacao Bed & Breakfast.
------------
Retrieving Results for Index 393: am timan.
Closest hotel is Palais du chef Canton Salamt.
------------
Retrieving Results for Index 397: salalah.
Closest hotel is HAMDAN PLAZA HOTEL SALALAH.
------------
Retrieving Results for Index 400: bara.
Missing field/result... skipping.
------------
Retrieving Results for Index 489: maceio.
Closest hotel is Hotel Ponta Verde Maceió.
------------
Retrieving Results for Index 491: pisco.
Closest hote

In [7]:
hotel_df

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
22,chabahar,25.2919,60.6430,70.34,46.0,0.0,5.64,IR,1.613081e+09,هتل گدروشیا
29,negombo,7.2083,79.8358,71.60,78.0,0.0,6.91,LK,1.613080e+09,Hotel Sunset Beach
50,saint-philippe,-21.3585,55.7679,78.80,65.0,0.0,5.75,RE,1.613080e+09,"Chambres d'hôte ""La Trinité"""
62,poum,-20.2333,164.0167,79.05,72.0,0.0,8.90,NC,1.613081e+09,Gîte kajeon
218,sao filipe,14.8961,-24.4956,72.07,67.0,0.0,9.91,CV,1.613081e+09,Tortuga B&B
249,vallenar,-28.5708,-70.7581,79.47,38.0,0.0,7.20,CL,1.613081e+09,Humacao Bed & Breakfast
393,am timan,11.0297,20.2827,73.33,18.0,0.0,7.11,TD,1.613081e+09,Palais du chef Canton Salamt
397,salalah,17.0151,54.0924,71.60,53.0,0.0,4.61,OM,1.613081e+09,HAMDAN PLAZA HOTEL SALALAH
400,bara,10.3744,10.7288,73.49,19.0,0.0,7.74,NG,1.613081e+09,NaN
489,maceio,-9.6658,-35.7353,77.00,83.0,0.0,5.75,BR,1.613081e+09,Hotel Ponta Verde Maceió


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Display figure

fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig



Figure(layout=FigureLayout(height='420px'))